# ****About:****
* The data I used is Cyclistic’s Historical Trip Data to analyze and identify trends.

* The previous 12 months data from 2021 March  to 2022 Feb  is used for analysis.

* The data is stored in CSV files. Each file contains one month data. Thus a total of 12 .csv files.

* The data is structured data ie., Organised data.

* The datasets have a different name because Cyclistic is a fictional company. For the purposes of this case study, the datasets are appropriate.**

In [36]:
import pandas as pd
from pandas.api.types import CategoricalDtype

import seaborn as sns

import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import cufflinks as cf

# Importing Necessary Python Libraries.

In [37]:
mar_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202103-divvy-tripdata.csv')
apr_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202104-divvy-tripdata.csv')
may_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202105-divvy-tripdata.csv')
jun_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202106-divvy-tripdata.csv')
jul_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202107-divvy-tripdata.csv')
aug_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202108-divvy-tripdata.csv')
sep_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202109-divvy-tripdata.csv')
oct_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202110-divvy-tripdata.csv')
nov_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202111-divvy-tripdata.csv')
dec_21 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202112-divvy-tripdata.csv')
jan_22 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202201-divvy-tripdata.csv')
feb_22 = pd.read_csv('/kaggle/input/cyclist-bike-sharecase-study-updated/Cyclistic Data/202202-divvy-tripdata.csv')

# Importing 12 CSV files.

In [38]:
df = pd.concat([mar_21,apr_21,may_21,jun_21,jul_21,aug_21,sep_21,oct_21,nov_21,dec_21,jan_22,feb_22], ignore_index=True)

# Joining 12 CSV's into 1 and naming it as "df".

In [39]:
df

# Top 5 & bottom 5 rows of dataframe "df".

In [40]:
df.columns

# Data Cleaning

In [41]:
df.rename(columns={'rideable_type': 'bike_type','started_at':'start_time','ended_at':'end_time',
                  'member_casual':'user_type'},inplace = True)

df.columns

#Rename Some colunms

In [42]:
df.shape

In [43]:
df.info()

In [44]:
df['end_time'] = pd.to_datetime(df['end_time'])
df['start_time'] = pd.to_datetime(df['start_time'])

# Coverting datatype of "started_at" & "ended_at" columns to "datetime64" type.

df['ride_length'] = (df.end_time-df.start_time)/pd.Timedelta(minutes=1)
df['ride_length'] = df['ride_length'].astype('int')

# Creating New Column "ride_length" and changing its datatype to "int32"
# In this column, each row contains the difference between "starting time" and "ending time" columns in minutes.


In [45]:
df = df.drop(['start_station_name','start_station_id','end_station_name','end_station_id','start_lat','start_lng','end_lat','end_lng'],axis=1)

# Deleting Columns which are not relevant for my analysis.

In [46]:
df.sort_values('ride_length')

# Sorting Values by "ride_length" column in Ascending order.

# Its seen that many rows in some months contained negative values. 
# Such errors happened because the "ending time" is earlier than the "starting time" in their respective rows.

In [47]:
df.drop(df[df['ride_length']<1].index,inplace = True)


# Removing 80845 rows containing negative values & ride length less than 1 minute. 
# Any trips that were below 60 seconds in length are potentially false starts or users trying to re-dock a bike to ensure it was secure.

In [48]:
sns.boxplot(data = df, x ='user_type', y = 'ride_length', order = ['member', 'casual'])

# Boxplot of column "Ride Length" to see the distribution of data between Member and Casual Rider.

* Its seen that casual riders use bikes for **more duration** (might be returning bikes after days) than members.

In [49]:
df.isna().sum()
# Checking if Dataframe "df" contains any missing values.

In [50]:
df.duplicated().sum()
#Checking if Dataframe "df" contains any duplicate values.


In [51]:
df['ride_id'] = df['ride_id'].str.strip()
# Removing leading and trailing whitespaces in column "Ride Id".

df.duplicated(subset=['ride_id']).sum()
# View any duplicate rows.

In [52]:
df['Year'] = pd.to_datetime(df['start_time']).dt.year

cats1 = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
df['Month'] = pd.to_datetime(df['start_time']).dt.month_name()
df['Month'] = df['Month'].astype(CategoricalDtype(categories=cats1, ordered=False))

cats2 = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
df['Day_of_week'] = df['start_time'].dt.day_name()
df['Day_of_week'] = df['Day_of_week'].astype(CategoricalDtype(categories=cats2, ordered=False))


df['Hour'] = df['start_time'].dt.hour

# Creating new columns "year", "month", "day_of_week", "hour" and Converting datatypes.

In [53]:
df = df.astype({'ride_id' : 'string', 'bike_type' : 'category', 'user_type' : 'category','Month' : 'category', 'Day_of_week' : 'category' })

# Coverting datatypes of each columns.

In [54]:
df.info()

# Data Analyzing & Visualization

# No. of Rides in 1 year

In [55]:
pd.pivot_table(df,
              index = 'user_type',
              values = 'ride_id',
              aggfunc = ['count'],
              margins = True,
              margins_name = 'Total Count')

In [56]:
fig_1 = df.groupby('user_type', as_index=False).count()

px.bar(fig_1, y = 'user_type', x = 'ride_id', range_x = [0,000000],
        color = 'user_type', 
        height = 300,
        text = 'ride_id', 
        labels = {'ride_id': 'No. of Rides', 'user_type': 'Member/Casual'},
        hover_name = 'user_type', hover_data = {'user_type': False, 'Month': False, 'ride_id': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

# Number of Rides in Each Month

In [57]:
pv2 = pd.pivot_table(df,index =['Year','Month','user_type'],
              values = 'ride_id',
              aggfunc = ['count'],
              margins = True,
              margins_name = 'Total_Count')
pv2 = pv2.loc[(pv2 !=0).any(axis=1)]
pv2

In [58]:
fig_2 = df.groupby(['Year', 'Month', 'user_type'], as_index=False).count()
fig_2 = fig_2[fig_2['ride_length'] != 0]

px.line(fig_2, x = 'Month', y = 'ride_id', range_y = [0,450000],
        color = 'user_type', 
        line_shape = 'spline',
        markers=True, 
        labels = {'ride_id': 'No_of_rides', 'Month': 'Months (Mar 2021 - Feb 2022)', 'user_type': 'Member/Casual'},
        hover_name = 'user_type', hover_data = {'user_type': False, 'Month': True, 'ride_id': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

* It shows that the total number of rides fall during Nov - Mar and rise during May-Oct, and peak at Jul-Aug.
* The behaviour of casual riders and members tend to be the same as the season changes.
* Maximum riders are using bikes in Jul - Aug.
* Casual riders overtake members during Jul.

# Number of Rides in Each Weekday

In [59]:
pd.pivot_table(df,
        index= ['Day_of_week', 'user_type'],
        values= 'ride_id',
        aggfunc = ['count'],
        margins = True)

In [60]:
fig_3 = df.groupby(['Day_of_week', 'user_type'],as_index=False).count()

px.line(fig_3, x='Day_of_week', y='ride_id',range_y = [0,45000],
       color = 'user_type',line_shape = 'spline',
        markers=True, 
        labels = {'ride_id': 'No. of Rides', 'user_type': 'Member/Casual', 'bike_type' : 'Bike Type'},
        hover_name = 'user_type', hover_data = {'user_type': False, 'ride_length': False}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})
       

* Here it shows more casual riders are using bike share on Weekends (ie., Saturdays and Sundays).
* But there are a fixed number of casual riders using on Weekdays, might be commuting.
* While the number of members riding tend to be same almost daily.

# Number of Rides in Each Hour

In [61]:
pd.pivot_table(df,index = ['Hour','user_type'],
              values = 'ride_id',
              aggfunc =['count'],
              margins = True,
              margins_name = 'Total_count')

In [62]:
fig_4 =  df.groupby(['Hour', 'user_type'],as_index=False).count()

px.line(fig_4, x = 'Hour',y = 'ride_id',color = 'user_type',
       line_shape='spline', markers =True, 
        labels = {'ride_id': 'No. of Rides', 'user_type': 'Member/Casual', 'bike_type' : 'Bike Type'},
        hover_name = 'user_type', hover_data = {'user_type': False, 'ride_length': False}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

* In a day, casual riders and members use bike share more during afternoon, peak use during evening.
* While in the morning time, the number of casual riders are way less than the members.

# Average Ride Length in 1 Year

In [63]:
pd.pivot_table(df,
               index = 'user_type',
               values = ['ride_length'],
               aggfunc = ['mean'],
               margins = True,
               margins_name = 'Total Ride Average')

In [64]:
fig_5 = df.groupby([ 'user_type'],as_index=False).mean()

px.bar (fig_5, y='user_type',x='ride_length',color = 'user_type',range_x = [0,35],
       text = 'ride_length',height=300,
       labels = {'ride_length': 'Average Ride Length (Min)', 'user_type': 'Member/Casual'},
       color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'},
       hover_name = 'user_type', hover_data = {'user_type': False, 'ride_length': True})

* The average ride length of casual riders are more than twice of members.

# Average Ride Length in Each Month

In [65]:
pd.pivot_table(df, index = ['Year','Month','user_type'],
              values = ['ride_length'],
              aggfunc = ['mean'],
              margins = True,
              margins_name = 'Avg Ride Length')

In [66]:
fig_6 = round(df.groupby(['Month', 'user_type'],as_index =False).mean(),2)

px.bar(fig_6, x='Month', y = 'ride_length',color = 'user_type',barmode ='group',
      range_y = [0,50],
      text ='ride_length', 
        labels = {'ride_length': 'Avg ride length(Min)', 'user_type': 'Member/Casual', 'bike_type' : 'Bike Type'},
        hover_name = 'user_type', hover_data = {'user_type': False, 'ride_length': False}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

* Average ride length of casual riders are more than twice than members in all months.

# Average Ride Length in each WeekDay

In [67]:
pd.pivot_table(df,index = ['Day_of_week', 'user_type'],
              values = 'ride_length',
              aggfunc = ['mean'],
              margins = True,
               margins_name = 'Total Ride Average')

In [68]:
fig_7 = round(df.groupby(['Day_of_week', 'user_type'],as_index =False).mean(),2)

px.bar(fig_7,x='Day_of_week',y = 'ride_length', color= 'user_type',
      barmode='group',range_y = [0,50],
      text = 'ride_length', 
      labels = {'ride_length': 'Avg length 0f Rides(Min)', 'user_type': 'Member/Casual', 'bike_type' : 'Bike Type'},
        hover_name = 'user_type', hover_data = {'user_type': False, 'ride_length': False}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

* In Weekends casual riders' ride length is maximum when compared to Weekdays.
* Members' ride length tend to be almost same in all Weekdays and marginally higher in Weekends.
* Both Casual riders and Members use bikes for long rides during Weekends.

# Bike Type Usage Between Casual riders and Members

In [69]:
pd.pivot_table(df,
               index = ['bike_type', 'user_type'],
               values = ['ride_id'],
               aggfunc = ['count'],
               margins = True,
               margins_name = 'Total Rides')

In [70]:
fig_8 = df.groupby(['bike_type', 'user_type'],as_index = False).count()
px.bar( fig_8, x='bike_type' , y = 'ride_id',color = 'user_type',range_y = [0,3000000],
       barmode = 'group'  , text = 'ride_id', 
        labels = {'ride_id': 'No. of Rides', 'user_type': 'Member/Casual', 'bike_type' : 'Bike Type'},
        hover_name = 'user_type', hover_data = {'user_type': False, 'ride_length': False}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})


# Conclusion
* Annual members and Casual riders use Cyclistic bike share differently.

* The average ride length of causual riders are more than twice as of members.

* From the average ride length difference, we can conclude that Annual members usually use bike share for daily commuting, while casual riders mostly use bike share for leisure rides mostly during Weekends.

* But there are a fixed number of casual riders who use bike share for commuting.

# Additional Data to expand the findings
* If the personally identifiable information and financial information were available, I could have calculated whether the casual riders had spent more money than if they opt for taking annual memberships.

# Recommendations
* A **new Annual Membership package for Weekend usage only** will attract current Weekend casual riders.

* **Promotions** aiming at **current Weekday casual riders** must be implemented as soon as possible. Those promtions must include the financial savings of taking membership when compared to single passes and full day passes for a year long period.

* A **Loyalty Program** for casual riders can be implemented, where **occasional membership fees discounts** must be given to casual riders with **high loyalty points**.